In [2]:
import requests
import pandas as pd
import numpy as np
from itertools import product

In [3]:
BASE = "https://www150.statcan.gc.ca/t1/wds/rest"

In [4]:
def getCubeMetadata(pid):
    url = f'{BASE}/getCubeMetadata'
    payload = [{"productId": pid}]
    response = requests.post(url, json=payload)
    response.raise_for_status()
    
    data = response.json()
    if not isinstance(data, list) or not data:
        raise ValueError(f"Unexpected payload: {data}")
        
    item = data[0]
    if item.get("status") != "SUCCESS":
        # WDS-level error even though HTTP was 200
        raise RuntimeError(f"WDS error: {item.get('status')} | {item.get('object')}")
    return item["object"]

In [5]:
def previewDimensions(pid, target="names", dimName=None):
    meta = getCubeMetadata(pid)
    dimensions = {
        dim["dimensionNameEn"]: {
            "position": dim["dimensionPositionId"], 
            "values": { m["memberNameEn"]: m["memberId"] for m in dim["member"] }
        } for dim in meta["dimension"]
    }

    if target == "full":
        print(dimensions)
    elif target == "names":
        dim_names = {dim: dimensions[dim].get("position") for dim in dimensions}
        print(dim_names)
    elif target == "values":
        if dimName is None or dimName not in dimensions.keys():
            raise ValueError(f"{dimName} is a valid dimension.")
        for name,dim in dimensions.items():
            if name == dimName:
                print(dim["values"])
                break

In [6]:
def buildCoordinates(pid, series_coords):
    # Get table metadata
    meta = getCubeMetadata(pid)

    # Build dimension mapping
    dim_mapping = {
        dim["dimensionNameEn"]: {
            "position": dim["dimensionPositionId"], 
            "values": { m["memberNameEn"]: m["memberId"] for m in dim["member"] }
        } for dim in meta["dimension"]
    }

    # Mapp column names and values to coordinate
    coordinates = []
    for series in series_coords:
        coords = ['0']*10
        for k,v in series.items():
            dim = dim_mapping.get(k)
            if dim is None:
                print(f"Warning: Could not locate dimension '{k}'")
                coords = None
                break
            pos = dim["position"]
            value = dim["values"].get(v)
            if value is None:
                print(f"Warning: No '{v}' found for the dimension '{k}'")
                coords = None
                break
            coords[int(pos)-1] = str(value)
        if coords:
            coordinates.append(".".join(coords))
    
    return coordinates, { dim: int(v["position"]) for dim,v in dim_mapping.items() }  

In [7]:
def expand_specs(series_spec):
    pairs = []
    for dim in series_spec:
        (k,v), = dim.items()
        vals = v if isinstance(v, (list, tuple)) else [v]
        pairs.append((k, list(vals)))
    keys = [k for k,_ in pairs]
    value_lists = [vals for _,vals in pairs]
    return [dict(zip(keys, combo)) for combo in product(*value_lists)]

In [22]:
def getVectorIds(pid, coords):
    # Translate coordiantes to vectors
    if len(coords) > 0:
        vec_payload = [{"productId": pid, "coordinate": c} for c in coords]
    else:
        raise Exception("Invalid coordinates. Please specify all required dimensions")
    series = requests.post(f"{BASE}/getSeriesInfoFromCubePidCoord", json=vec_payload).json()
    vec_map = { s["object"]["vectorId"]: s["object"]["SeriesTitleEn"] for s in series if s["object"]["vectorId"] != 0 }

    if not vec_map:
        raise Exception(f"Failed to retrieve vectors for coordinates: {coords}. Please specify all required dimensions")

    return vec_map

In [25]:
def getTableData(pid, series_specs, startRefPerid="2000-01-01", endRefPeriod="2025-12-31"):
    expanded_specs = expand_specs(series_specs)
    
    # Build coordinates based on series specification
    coords, dim_map = buildCoordinates(pid, expanded_specs)
    
    # Get vectors
    vec_map = getVectorIds(pid, coords)

    vectorIds = ",".join([ f'"{v}"' for v in vec_map.keys()])
    series = requests.get(
        f"{BASE}/getDataFromVectorByReferencePeriodRange?vectorIds={vectorIds}&startRefPeriod={startRefPerid}&endReferencePeriod={endRefPeriod}"
    ).json()

    final_df = None

    for s in series:
        vId = s["object"].get("vectorId")
        index_cols = [list(spec.keys())[0] for spec in series_specs]
        index_cols.sort(key=lambda col: dim_map.get(col, float('inf')))
        index_vals = vec_map[vId].split(";")
        row_index = {k:v for k,v in zip(index_cols, index_vals) if k in dim_map.keys()}
        
        dataPoints = s["object"].get("vectorDataPoint")
        rows = []
        for pt in dataPoints:
            value = pt["value"]
            ref_date = pt["refPer"]
            row = row_index | { "REF_DATE": ref_date, "VALUE": value }
            rows.append(row)
        df = pd.DataFrame(rows)
        final_df = pd.concat([final_df, df], ignore_index=True)

    return final_df

## Tests

In [10]:
previewDimensions(pid=14100287, target="full")

{'Geography': {'position': 1, 'values': {'Canada': 1, 'Newfoundland and Labrador': 2, 'Prince Edward Island': 3, 'Nova Scotia': 4, 'New Brunswick': 5, 'Quebec': 6, 'Ontario': 7, 'Manitoba': 8, 'Saskatchewan': 9, 'Alberta': 10, 'British Columbia': 11}}, 'Labour force characteristics': {'position': 2, 'values': {'Population': 1, 'Labour force': 2, 'Employment': 3, 'Full-time employment': 4, 'Part-time employment': 5, 'Unemployment': 6, 'Unemployment rate': 7, 'Participation rate': 8, 'Employment rate': 9}}, 'Gender': {'position': 3, 'values': {'Total - Gender': 1, 'Men+': 2, 'Women+': 3}}, 'Age group': {'position': 4, 'values': {'15 years and over': 1, '15 to 64 years': 8, '15 to 24 years': 2, '15 to 19 years': 3, '20 to 24 years': 4, '25 years and over': 5, '25 to 54 years': 6, '55 years and over': 7, '55 to 64 years': 9}}, 'Statistics': {'position': 5, 'values': {'Estimate': 1, 'Standard error of estimate': 2, 'Standard error of month-to-month change': 3, 'Standard error of year-over-y

In [11]:
previewDimensions(pid=14100287, target="names")

{'Geography': 1, 'Labour force characteristics': 2, 'Gender': 3, 'Age group': 4, 'Statistics': 5, 'Data type': 6}


In [12]:
previewDimensions(pid=14100287, target="values", dimName="Geography")

{'Canada': 1, 'Newfoundland and Labrador': 2, 'Prince Edward Island': 3, 'Nova Scotia': 4, 'New Brunswick': 5, 'Quebec': 6, 'Ontario': 7, 'Manitoba': 8, 'Saskatchewan': 9, 'Alberta': 10, 'British Columbia': 11}


In [13]:
labour_data_specs = [
    {"Geography": ["Quebec"]},
    {"Labour force characteristics": ["Employment rate", "Unemployment rate"]},
    {"Data type": ["Seasonally adjusted"]},
    {"Statistics": ["Estimate"]},
    {"Gender": ["Total - Gender"]},
    {"Age group": "15 years and over"}
]

In [14]:
%time
labour_tab = getTableData(pid=14100287, series_specs=labour_data_specs, startRefPerid="2000-01-01", endRefPeriod="2025-12-31")

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
labour_tab

,Geography,Labour force characteristics,Gender,Age group,Statistics,Data type,REF_DATE,VALUE
0,Quebec,Unemployment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2000-01-01,8.2
1,Quebec,Unemployment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2000-02-01,8.5
2,Quebec,Unemployment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2000-03-01,8.5
3,Quebec,Unemployment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2000-04-01,8.6
4,Quebec,Unemployment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2000-05-01,8.3
...,...,...,...,...,...,...,...,...
609,Quebec,Employment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2025-03-01,61.3
610,Quebec,Employment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2025-04-01,61.4
611,Quebec,Employment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2025-05-01,61.1
612,Quebec,Employment rate,Total - Gender,15 years and over,Estimate,Seasonally adjusted,2025-06-01,61.3
